## Importing Libraries

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
plt.style.use('dark_background')

In [ ]:
df = pd.read_csv('../input/engineering-graduate-salary-prediction/Engineering_graduate_salary.csv')
df.head()

In [ ]:
df.shape

In [ ]:
df.columns

## Dropping Few Columns 

In [ ]:
df.drop(['ID', 'DOB', 'CollegeID', '12graduation' ,'10board', '12board' , 'CollegeState'
                                     ,'CollegeCityID', 'CollegeCityTier'
                                         ,'GraduationYear'], axis = 1, inplace = True)
df.head()

In [ ]:
df = df.drop_duplicates()

In [ ]:
df.info()

In [ ]:
df.describe()

In [ ]:
df.head()

In [ ]:
df.Degree.unique()

## Working on Specialization Column

In [ ]:
df.Specialization.unique()

In [ ]:
df.Specialization.value_counts()

In [ ]:
specialization = df.Specialization.value_counts(ascending = False)
specialization

In [ ]:
specializationlessthan10 = specialization[specialization<=10]
specializationlessthan10

In [ ]:
def removespeciallessthan10(value):
    if value in specializationlessthan10:
        return 'other'
    else:
        return value
df.Specialization = df.Specialization.apply(removespeciallessthan10)
df.Specialization.unique()

In [ ]:
df.head()

## Working on GPA Column

In [ ]:
plt.scatter(df.index, df['collegeGPA'])

In [ ]:
df = df[(df['collegeGPA']>40)]
df.shape

## Visualizing English, Logical and Quant

In [ ]:
plt.figure(figsize = (10,10))
plt.subplot(2,2,1)
plt.scatter(df.index, df.English)
plt.title('English')

plt.subplot(2,2,2)
plt.scatter(df.index, df.Logical)
plt.title('Logical')

plt.subplot(2,2,3)
plt.scatter(df.index, df.Quant)
plt.title('Quant')

plt.show()

## Visualizing Few Other Columns

In [ ]:
df.head()

In [ ]:
df.columns

In [ ]:
plt.figure(figsize = (10,10))
plt.subplot(2,2,1)
plt.scatter(df.index, df.Domain)
plt.title('Domain')

plt.subplot(2,2,2)
plt.scatter(df.index, df.ComputerProgramming)
plt.title('ComputerProgramming')

plt.subplot(2,2,3)
plt.scatter(df.index, df.ElectronicsAndSemicon)
plt.title('ElectronicsAndSemicon')

plt.show()

## Replacing -1 with Null

In [ ]:
df = df.replace(-1, np.nan)

## Filling Null Values with Mean

In [ ]:
cols_with_nan = [column for column in df.columns if df.isna().sum()[column] > 0]
for column in cols_with_nan:
    df[column] = df[column].fillna(df[column].mean())

# **Data Visualization**

In [ ]:
df.head()

In [ ]:
sns.countplot(df.Gender, palette = 'inferno')

In [ ]:
sns.scatterplot(df['10percentage'], df['12percentage'])

In [ ]:
sns.scatterplot(df['10percentage'], df['12percentage'], hue = df.CollegeTier)

**10 Percentage can be dropped as 12 Percentage is related to it, so let's have only one**

In [ ]:
df.drop(['10percentage'], axis = 1, inplace = True)

In [ ]:
plt.figure(figsize = (10,6))
sns.countplot(df['Specialization'])
plt.xticks(rotation = 90)

In [ ]:
df.head()

## **Plotting GPA vs Salary**

In [ ]:
plt.figure(figsize = (15,8))
sns.scatterplot(df.collegeGPA, df.Salary, hue = df.Specialization, palette = 'inferno')

In [ ]:
plt.figure(figsize = (15,8))
sns.scatterplot(df.collegeGPA, df.Salary, hue = df.Degree, palette = 'inferno')

## Openness to Experience vs Salary

In [ ]:
plt.figure(figsize = (12,6))
sns.scatterplot(df.openess_to_experience, df.Salary)

## Domain vs Salary

In [ ]:
plt.figure(figsize = (12,6))
sns.scatterplot(df.Domain, df.Salary, hue = df.Degree, palette = 'inferno')

In [ ]:
plt.figure(figsize = (8,5))
sns.heatmap(df.corr())

In [ ]:
# sns.pairplot(df)

## **Data Preprocessing**

In [ ]:
df.head()

In [ ]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()

df.Gender = le.fit_transform(df.Gender)
df.Degree = le.fit_transform(df.Degree)
df.Specialization = le.fit_transform(df.Specialization)

df.head()

## **Creating XGBoost Model**

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

from xgboost import XGBRegressor

In [ ]:
x = df.drop('Salary', axis=1)
y = df['Salary']

sc = StandardScaler()
x = sc.fit_transform(x)

X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.2, shuffle=True, random_state=0)

In [ ]:
xgb = XGBRegressor()
xgb.fit(X_train, y_train)

## **Predicting X_test Data**

In [ ]:
predictions = xgb.predict(X_test)

In [ ]:
diff = y_test - predictions

## **Visualizing Difference**

In [ ]:
plt.figure(figsize = (12,6))
sns.distplot(diff)

## **Checking R^2 Score**

In [ ]:
xgb_r2_score = xgb.score(X_test, y_test)

print("XGBoost R2 Score: ", (xgb_r2_score))